# Table des Matières
1. [Introduction et Sélection des données](#intro)
2. [Exploration et traitement des données](#traitement)
3. [Modélisation et évaluation](#model)
4. [Comunication des résultats](#comm)
5. [Retour d'expérience](#retour) 

  
 

---

## Introduction et sélection des données <a class="anchor" id="intro"></a>

Dans le cadre de notre projet de machine Learning, nous allons traiter un sujet d'apprentissage supervisé à partir de données ouvertes. Tout au long de notre projet, nous utiliserons l'IA générative (ChatGPT) pour nous aider dans notre travail. 

Dans un premier temps, nous commençons par demander à ChatGPT des idées de sujets.

Jeu de données ML généré par l'IA :
- DPE (Analyse énergétique, classification des logements)
- Transports en commun (prédiction affluence, optimisation trajet)
- Pollution (prédiction des pics de pollution, classification zones polluées)
- Reconnaissance d'image
- Traitement du langage naturel
- Donnée de santé
- Donnée financière économique (prédiction de tendance..)

<u>Analyse critique de l'IA :</u>

L'IA propose souvent des sujets très classiques, déjà réalisé. Il propose même des projets déjà tout fait sur Kaggle. Il manque d'originalité ! 

### Données initiales <a class="anchor" id="intro"></a>

<u>Changement de problématique</u>

Initialement de notre projet portait sur la prédiction des passoires thermiques à partir des données DPE : **Quels logements sont susceptibles d’être classés comme passoires thermiques (classe F ou G) ?** 

Cependant, après une analyse approfondie des données, nous avons décidé de changer de jeu de données et de problématique.

La problématique initiale, s'est révélée trop simple pour un sujet de machine learning. Les données DPE sont directement corrélées avec les passoires thermiques, ce qui limite les défis en termes de modélisation et d'analyse.

Nous avons donc opté pour une nouvelle problématique, avec un jeu de données moins binaires. 


<u>Problématique générée avec L'IA sur les données DPE</u>

- Peut-on prédire la consommation énergétique d’un logement à partir de ses caractéristiques issues du DPE ?
- Quels logements sont susceptibles d’être classés comme passoires thermiques (classe F ou G) ?
- Peut-on prédire la classe énergétique (A à G) d’un logement à partir des caractéristiques fournies par le DPE ?

<u>Critique de l'IA sur notre approche précedente</u>

Votre approche est bien structurée et inclut des variables clés dans des domaines pertinents pour prédire si un logement est une passoire thermique. 

*Points positifs* :
Les variables sélectionnées correspondent bien à des déterminants majeurs de la performance énergétique.
Vous incluez à la fois des variables quantitatives (ex. : consommation énergétique, déperditions) et qualitatives (ex. : type d'installation de chauffage, zone climatique), ce qui permet une modélisation riche.

*Critiques et suggestions* :
Les bases de données DPE peuvent contenir des valeurs manquantes ou aberrantes (ex. : surfaces non renseignées, consommations incohérentes).
Vérifiez les distributions des variables pour détecter des anomalies.
Plusieurs variables peuvent être fortement corrélées. Une analyse des corrélations peut être utile.
Les passoires thermiques (F et G) pourraient représenter une minorité des données. Cela peut biaiser l’entraînement du modèle.
Les bâtiments anciens sont souvent surreprésentés parmi les passoires thermiques, ce qui peut conduire le modèle à négliger d'autres facteurs. Une analyse exploratoire approfondie est nécessaire.
Les algorithmes robustes pour ce type de problème incluent :
 Forêts aléatoires (Random Forests): pour leur interprétabilité et leur capacité à gérer des variables mixtes.
 Gradient Boosting (XGBoost, LightGBM) : pour leur performance sur des ensembles déséquilibrés.
 Réseaux de neurones : si les données sont enrichies avec des caractéristiques complexes (géographiques, temporelles).

lien données DPE : https://drive.google.com/file/d/1nUbA6m3SQ9PXpc9i9tD6yADe0fDFYKTc/view?usp=drive_link


### Données définitives <a class="anchor" id="intro"></a>

Notre nouvelle problématique repose sur la prédiction du prix des billets d’avion. Le prix des billets d’avion sont influencés par une multitude de facteurs (dates de départ et de réservation, destinations, compagnies aériennes, saisonnalité, etc.), ce qui implique une plus grande diversité dans les données que notre sujet précedent. 

Afin de répondre à notre problématique, nous utilisons un jeu de données issu de vols aller simple du site Expedia entre le 16 avril 2022 et le 5 octobre 2022. 

Voici l'intégralité des variables présentes dans notre jeu de données : 

- **Identifiants et Dates**  
  - `legId` : Identifiant unique pour chaque vol.  
  - `searchDate` : Date de la recherche effectuée sur Expedia (AAAA-MM-JJ).  
  - `flightDate` : Date du vol (AAAA-MM-JJ).  

- **Informations sur les Aéroports**  
  - `startingAirport` : Code IATA (trois caractères) de l’aéroport de départ.  
  - `destinationAirport` : Code IATA (trois caractères) de l’aéroport d’arrivée.  

- **Tarification et Conditions**  
  - `fareBasisCode` : Code de base tarifaire.  
  - `baseFare` : Tarif de base du billet (en USD).  
  - `totalFare` : Tarif total incluant taxes et frais (en USD).  
  - `isBasicEconomy` : Indique si le billet appartient à la classe économique basique (booléen).  
  - `isRefundable` : Indique si le billet est remboursable (booléen).  

- **Caractéristiques du Vol**  
  - `isNonStop` : Indique si le vol est direct (booléen).  
  - `travelDuration` : Durée totale du trajet (heures et minutes).  
  - `elapsedDays` : Nombre de jours écoulés avant le vol.  
  - `seatsRemaining` : Nombre de sièges restants disponibles.  
  - `totalTravelDistance` : Distance totale parcourue en miles.  

- **Segments du Vol**  
Les données contiennent également des informations détaillées sur chaque segment du trajet :  
  - Heures de départ et d’arrivée (`segmentsDepartureTimeRaw`, `segmentsArrivalTimeRaw`).  
  - Codes aéroportuaires des départs et arrivées (`segmentsDepartureAirportCode`, `segmentsArrivalAirportCode`).  
  - Compagnies aériennes et avions utilisés (`segmentsAirlineName`, `segmentsEquipmentDescription`).  
  - Durée et distance de chaque segment (`segmentsDurationInSeconds`, `segmentsDistance`).  
  - Cabine (`segmentsCabinCode`).  


  Lien données Expédia : https://drive.google.com/file/d/1K_oI18zxBvjVMAJJhPKGhJpfeVdNPYJC/view?usp=drive_link

<u>Problématique générées par l'IA sur les données Expédia</u>

- Prévision des prix des vols
- Analyse des tendances saisonnières
- Classification des vols selon leur popularité
- Clustering des destinations
- Prédiction de la demande pour une destination
- Analyse des facteurs influençant le prix des vols

<u>Critique de l'IA sur notre approche</u>

*Points forts de l'approche :*

- Richesse des données : Votre jeu de données contient une grande diversité de variables couvrant plusieurs aspects essentiels (dates, informations sur les vols, tarifs, caractéristiques des segments, etc.). Cela donne une base solide pour entraîner un modèle robuste.

- Problématique claire : La prédiction des prix est un problème bien défini, et les variables disponibles semblent adaptées à cette tâche.

- Couverture des facteurs influents : Vous prenez en compte des aspects critiques comme les dates, les distances, les caractéristiques des vols (direct/non-stop, classe économique basique), ainsi que la demande (nombre de sièges restants).

*Axes d'amélioration et recommandations :*

- Analyse exploratoire des données (EDA) : Avant toute modélisation, il sera crucial d’effectuer une analyse approfondie pour détecter les valeurs manquantes ou anomalies, visualiser les distributions des variables, identifier les corrélations entre variables.
- Explorer les effets saisonniers liés à searchDate et flightDate.
- Préparation des données : certaines variables nécessiteront une transformation ou une ingénierie des fonctionnalités comme l'encodage des variables catégoriques, extraire le mois, le jour de la semaine.
- Choix et évaluation des modèles : Vous devez expérimenter plusieurs algorithmes pour identifier celui qui s'adapte le mieux à vos données.
- Utilisez des métriques pertinentes comme la RMSE ou le MAPE.
- Effectuez une validation croisée pour garantir la robustesse des résultats.



In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Modèles
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# Métriques
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Métriques
from sklearn.metrics import mean_squared_error, r2_score

In [47]:
df_flight = pd.read_csv("data/itineraries_sample.csv")

In [48]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000373 entries, 0 to 1000372
Data columns (total 27 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   legId                              1000373 non-null  object 
 1   searchDate                         1000373 non-null  object 
 2   flightDate                         1000373 non-null  object 
 3   startingAirport                    1000373 non-null  object 
 4   destinationAirport                 1000373 non-null  object 
 5   fareBasisCode                      1000373 non-null  object 
 6   travelDuration                     1000373 non-null  object 
 7   elapsedDays                        1000373 non-null  int64  
 8   isBasicEconomy                     1000373 non-null  bool   
 9   isRefundable                       1000373 non-null  bool   
 10  isNonStop                          1000373 non-null  bool   
 11  baseFare                

## Exploration et traitement des données <a class="anchor" id="traitement"></a>

Dans un premier temps, nous explorons notre jeu de données et réalisons quelques traitements. 

### Traitement des valeurs manquantes <a class="anchor" id="traitement"></a>

In [49]:
# Analyse de la proportion de valeurs manquantes
pourcentage_valeurs_manquantes = df_flight.isnull().mean() * 100
pourcentage_valeurs_manquantes

legId                                0.000000
searchDate                           0.000000
flightDate                           0.000000
startingAirport                      0.000000
destinationAirport                   0.000000
fareBasisCode                        0.000000
travelDuration                       0.000000
elapsedDays                          0.000000
isBasicEconomy                       0.000000
isRefundable                         0.000000
isNonStop                            0.000000
baseFare                             0.000000
totalFare                            0.000000
seatsRemaining                       0.000000
totalTravelDistance                  7.416434
segmentsDepartureTimeEpochSeconds    0.000000
segmentsDepartureTimeRaw             0.000000
segmentsArrivalTimeEpochSeconds      0.000000
segmentsArrivalTimeRaw               0.000000
segmentsArrivalAirportCode           0.000000
segmentsDepartureAirportCode         0.000000
segmentsAirlineName               

In [50]:
# Nous constatons que notre jeu de données contient peu de valeurs manquantes et sur peu de variables différentes.
# Notre jeu de données étant assez conséquent, nous décidons de supprimer les valeurs les lignes contenant au moins une valeur manquante. 

# Retirer toutes les lignes contenant des valeurs manquantes
df_cleaned = df_flight.dropna()

# Vérifier que les valeurs manquantes ont été supprimées
print(df_cleaned.isnull().sum())

legId                                0
searchDate                           0
flightDate                           0
startingAirport                      0
destinationAirport                   0
fareBasisCode                        0
travelDuration                       0
elapsedDays                          0
isBasicEconomy                       0
isRefundable                         0
isNonStop                            0
baseFare                             0
totalFare                            0
seatsRemaining                       0
totalTravelDistance                  0
segmentsDepartureTimeEpochSeconds    0
segmentsDepartureTimeRaw             0
segmentsArrivalTimeEpochSeconds      0
segmentsArrivalTimeRaw               0
segmentsArrivalAirportCode           0
segmentsDepartureAirportCode         0
segmentsAirlineName                  0
segmentsAirlineCode                  0
segmentsEquipmentDescription         0
segmentsDurationInSeconds            0
segmentsDistance         

### Exploration des données : Statistiques descriptives <a class="anchor" id="traitement"></a>

In [51]:
def descriptive_statistics(df):
    """
    Effectue des statistiques descriptives sur toutes les variables d'un DataFrame.
    Sépare les variables quantitatives et qualitatives.
    """
    # Identifier les types de variables
    quali_vars = df.select_dtypes(include=['object', 'category']).columns
    quanti_vars = df.select_dtypes(include=['number']).columns

    # Résultats pour les variables quantitatives
    if not quanti_vars.empty:
        print("\nStatistiques descriptives pour les variables quantitatives :")
        print(df[quanti_vars].describe().transpose())

    # Résultats pour les variables qualitatives
    if not quali_vars.empty:
        print("\nStatistiques descriptives pour les variables qualitatives :")
        for col in quali_vars:
            print(f"\nVariable qualitative : {col}")
            print(df[col].value_counts())
            print(f"Valeurs uniques : {df[col].nunique()}")

descriptive_statistics(df_cleaned)


Statistiques descriptives pour les variables quantitatives :
                        count         mean         std    min     25%  \
elapsedDays          910463.0     0.146411    0.353521   0.00    0.00   
baseFare             910463.0   305.493268  182.323163  13.37  171.16   
totalFare            910463.0   350.829351  198.045073  28.97  203.60   
seatsRemaining       910463.0     6.396857    2.495220   0.00    5.00   
totalTravelDistance  910463.0  1613.705556  858.523345  89.00  891.00   

                         50%      75%     max  
elapsedDays             0.00     0.00     2.0  
baseFare              273.49   408.37  7000.0  
totalFare             316.10   465.10  7548.6  
seatsRemaining          7.00     9.00    10.0  
totalTravelDistance  1468.00  2417.00  4498.0  

Statistiques descriptives pour les variables qualitatives :

Variable qualitative : legId
legId
a500f6e457545a62dc0e23d7f1219395    7
ee81fec87daaf91bea92fad86e6245f0    6
38dd3ce4b9e2ff2d3cb116658313f87d    6


### Sélection des données <a class="anchor" id="traitement"></a>

Après avoir analysé nos données, nous sélectionnons les variables qui semblent pertinantes. 

Voici la liste des variables que nous retenons : 
- `legId`
- `searchDate`
- `flightDate`
- `startingAirport`
- `destinationAirport`
- `fareBasisCode`
- `travelDuration`
- `elapsedDays`
- `isBasicEconomy`
- `isRefundable`
- `isNonStop`
- `totalFare`
- `seatsRemaining`
- `totalTravelDistance`
- `segmentsAirlineName`


In [52]:
var_select = [
'legId',
'searchDate',
'flightDate',
'startingAirport',
'destinationAirport',
'fareBasisCode',
'travelDuration',
'elapsedDays',
'isBasicEconomy',
'isRefundable',
'isNonStop',
'totalFare',
'seatsRemaining',
'totalTravelDistance',
'segmentsAirlineName'
]

In [53]:
df_cleaned = df_cleaned[var_select]
df_cleaned

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName
1,6203bbd77fbd8e40021ee3e88ffa9edc,2022-04-16,2022-04-17,ATL,IAD,KA0NA0MC,PT12H22M,0,False,False,False,746.14,2,1224.0,Delta||Cape Air||Cape Air||United
2,34bb71c85bd77485193f5d83c553d783,2022-04-16,2022-04-17,ATL,JFK,EAA0OKEN,PT9H15M,0,False,False,False,672.19,1,762.0,United||Delta
3,691ae27539fcaab7ea209c67d36a6bdb,2022-04-16,2022-04-17,ATL,LAX,V0AGZNN1,PT7H47M,0,False,False,False,370.60,4,1954.0,American Airlines||American Airlines
4,5acd2ad9bbc5dfcdbfb2d70920c45a55,2022-04-16,2022-04-17,ATL,LAX,MA0QA0MQ,PT4H57M,0,False,False,True,598.61,7,1943.0,Delta
5,48250c83294e10c36c992ec208fc62f7,2022-04-16,2022-04-17,ATL,LGA,KA0NX0MQ,PT2H12M,0,False,False,True,244.60,1,762.0,Delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000368,9877500f9069b28d3bbde94206bb48c0,2022-10-05,2022-11-12,DFW,MIA,VUAWZNN1,PT6H55M,0,False,False,False,244.60,7,2100.0,American Airlines||American Airlines
1000369,75f5c5422edf8d0074b624106170fc2f,2022-10-05,2022-11-12,DFW,OAK,GH4OAJBN,PT7H42M,0,True,False,False,152.60,7,2335.0,Alaska Airlines||Alaska Airlines
1000370,b48181202b764fa7fddbf23823fa2903,2022-10-05,2022-11-12,DFW,PHL,TAVOA0BG,PT6H29M,0,True,False,False,158.60,9,1392.0,Delta||Delta
1000371,5ed2af184418e2a2f67d2839a593c1e9,2022-10-05,2022-11-12,DFW,PHL,QUAIXSM3,PT9H49M,1,False,False,False,463.20,7,1304.0,American Airlines||American Airlines


### Création de nouvelles variables <a class="anchor" id="traitement"></a>

Enfin, nous créons de nouvelles variables à partir des variables existantes (feature engineering). 

Ce processus permet d'améliorer les performances et l'apprentissage des modèles de machine learning.
Les nouvelles variables permettent de mieux représenter les facteurs influençant le prix des billets. Elles enrichissent les informations et facilitent la compréhension des relations entre les variables.  

Voici la liste des variables créées :

- `daysBetweenSearchAndFlight`: Nombre de jours entre la recherche du billet et le vol.  
   *Intérêt* : Capturer l’effet de l’anticipation, qui influence souvent les prix (réservations de dernière minute souvent plus coûteuses).  

- `searchDayOfWeek` et `flightDayOfWeek` : Jour de la semaine pour la recherche et le vol (0 = lundi, ..., 6 = dimanche).  
   *Intérêt* : Les prix peuvent varier selon le jour de la semaine, avec des tarifs souvent plus élevés les week-ends ou en début de semaine pour les vols d'affaires.  

- `searchMonth`, `flightMonth`, `flightYear` : Mois de la recherche, mois et année du vol.  
   *Intérêt* : Ces variables permettent de capturer la saisonnalité des prix (vacances scolaires, haute/basse saison touristique).  

- `flightWeekend`: Indique si le vol a lieu un samedi ou un dimanche (booléen).  
   *Intérêt* : Les vols du week-end sont souvent soumis à une demande plus élevée pour les voyages personnels, influençant les prix.  

- `travelDurationMinutes`: Durée totale du vol en minutes, convertie depuis un format ISO 8601 (`PT4H57M` -> 297 minutes).  
   *Intérêt* : La durée est directement liée au coût opérationnel du vol, qui influence les prix.  

- `fareType`: Classe tarifaire probable basée sur la première lettre de `fareBasisCode` (First, Business, Premium Economy, Economy).  
   *Intérêt* : Les classes de cabine reflètent des niveaux de service différents et des stratégies tarifaires spécifiques.  

- `isOvernightFlight`: Indique si le vol inclut une nuitée (déduit de `elapsedDays > 0`).  
   *Intérêt* : Les vols avec nuitée, souvent pour les trajets internationaux ou avec escales, ont des prix généralement plus élevés.  

- `distanceBands`: Catégorie de distance basée sur `totalTravelDistance` (short-haul <500 miles, medium-haul 500-1500 miles, long-haul >1500 miles)
   *Intérêt* : Les prix varient selon les distances, avec des structures tarifaires différentes pour les courts, moyens, et longs trajets. 

- **`firstAirline`** : Nom de la première compagnie aérienne, extrait de `segmentsAirlineName` (si plusieurs, seul le premier nom est pris).  
  *Intérêt* : Les compagnies aériennes ont des politiques tarifaires différentes, et certaines sont plus coûteuses que d’autres. 

In [54]:
# ===============================
# 0) Définition d'une fonction
#    pour parser la durée ISO 8601
# ===============================
def parse_iso8601_duration(duration_str):
    """
    Convertit une durée au format ISO 8601 (ex: 'PT4H57M') 
    en un nombre entier de minutes.
    Si la chaîne est invalide ou NaN, renvoie np.nan.
    """
    if pd.isnull(duration_str):
        return np.nan
    
    # Pattern simplifié pour extraire heures (H) et minutes (M), ex: PT4H57M
    pattern = r'^PT(?:(\d+)H)?(?:(\d+)M)?$'
    match = re.match(pattern, duration_str.strip())
    if not match:
        return np.nan  # Format non conforme ou autre
    
    hours_str, minutes_str = match.groups()
    hours = int(hours_str) if hours_str else 0
    minutes = int(minutes_str) if minutes_str else 0
    
    total_minutes = hours * 60 + minutes
    return total_minutes

# ===============================
# 1) Conversion des dates
# ===============================
df_cleaned['searchDate'] = pd.to_datetime(df_cleaned['searchDate'], errors='coerce')
df_cleaned['flightDate'] = pd.to_datetime(df_cleaned['flightDate'], errors='coerce')

# ===============================
# 2) daysBetweenSearchAndFlight
#    (flightDate - searchDate) en jours
# ===============================
df_cleaned['daysBetweenSearchAndFlight'] = (df_cleaned['flightDate'] - df_cleaned['searchDate']).dt.days

# ===============================
# 3) searchDayOfWeek et flightDayOfWeek
#    (0 = Lundi, ..., 6 = Dimanche)
# ===============================
df_cleaned['searchDayOfWeek'] = df_cleaned['searchDate'].dt.dayofweek
df_cleaned['flightDayOfWeek'] = df_cleaned['flightDate'].dt.dayofweek

# ===============================
# 4) searchMonth, flightMonth, flightYear
# ===============================
df_cleaned['searchMonth'] = df_cleaned['searchDate'].dt.month
df_cleaned['flightMonth'] = df_cleaned['flightDate'].dt.month
df_cleaned['flightYear'] = df_cleaned['flightDate'].dt.year

# ===============================
# 5) flightWeekend (booléen)
#    True si Samedi ou Dimanche
# ===============================
df_cleaned['flightWeekend'] = df_cleaned['flightDayOfWeek'].isin([5, 6])

# ===============================
# 6) Conversion de travelDuration
#    (ex: 'PT4H57M' -> minutes int)
# ===============================
df_cleaned['travelDurationMinutes'] = df_cleaned['travelDuration'].apply(parse_iso8601_duration)

# ===============================
# 7) fareType
#    Basé sur la 1ère lettre de fareBasisCode
# ===============================
FIRST = {'F', 'A', 'P'}
BUSINESS = {'C', 'J', 'D', 'I', 'Z'}
PREMIUM_ECONOMY = {'W', 'S', 'R'}
ECONOMY = {'Y', 'B', 'M', 'U', 'H', 'Q', 'K', 'L', 'G', 'V', 'T', 'N', 'X', 'O', 'E'}

def map_fare_type(code):
    """
    Retourne la cabine probable (First, Business, Premium Economy, Economy, Unknown)
    en se basant sur la première lettre du fareBasisCode.
    """
    if pd.isnull(code) or not code:
        return 'Unknown'
    
    first_char = str(code).strip().upper()[0]
    
    if first_char in FIRST:
        return 'First'
    elif first_char in BUSINESS:
        return 'Business'
    elif first_char in PREMIUM_ECONOMY:
        return 'Premium Economy'
    elif first_char in ECONOMY:
        return 'Economy'
    else:
        return 'Unknown'

df_cleaned['fareType'] = df_cleaned['fareBasisCode'].apply(map_fare_type)

# ===============================
# 8) isOvernightFlight
#    Si elapsedDays > 0
# ===============================
df_cleaned['isOvernightFlight'] = df_cleaned['elapsedDays'] > 0

# ===============================
# 9) distanceBands
#    short-haul (<500), medium-haul (<=1500), long-haul (>1500)
# ===============================
def define_distance_band(x):
    if pd.isnull(x):
        return 'unknown'
    elif x < 500:
        return 'short-haul'
    elif x <= 1500:
        return 'medium-haul'
    else:
        return 'long-haul'

df_cleaned['distanceBands'] = df_cleaned['totalTravelDistance'].apply(define_distance_band)

# ===============================
# 10) Extraction du premier nom de compagnie aérienne
#    Si segmentsAirlineName contient des valeurs séparées par '||', on prend la première
# ===============================
def extract_first_airline(airline_name):
    if pd.isnull(airline_name):
        return np.nan  # Retourne NaN si la valeur est manquante
    return airline_name.split('||')[0]  # Prend la première partie de la chaîne séparée par '||'

df_cleaned['firstAirline'] = df_cleaned['segmentsAirlineName'].apply(extract_first_airline)

# ===============================
# 12) Constitution de df_final
#     Avec les features finales + la cible
# ===============================
df_final = df_cleaned[[
    'daysBetweenSearchAndFlight', 
    'startingAirport',
    'destinationAirport',
    'searchMonth', 
    'searchDayOfWeek',
    'flightYear', 
    'flightMonth', 
    'flightDayOfWeek', 
    'flightWeekend', 
    'fareType',
    'elapsedDays', 
    'isOvernightFlight',
    'totalTravelDistance', 
    'distanceBands',
    'travelDurationMinutes',     
    #'firstAirline',              
    'totalFare'
]]

# Vérification
df_final.head()


,daysBetweenSearchAndFlight,startingAirport,destinationAirport,searchMonth,searchDayOfWeek,flightYear,flightMonth,flightDayOfWeek,flightWeekend,fareType,elapsedDays,isOvernightFlight,totalTravelDistance,distanceBands,travelDurationMinutes,totalFare
1,1,ATL,IAD,4,5,2022,4,6,True,Economy,0,False,1224.0,medium-haul,742.0,746.14
2,1,ATL,JFK,4,5,2022,4,6,True,Economy,0,False,762.0,medium-haul,555.0,672.19
3,1,ATL,LAX,4,5,2022,4,6,True,Economy,0,False,1954.0,long-haul,467.0,370.60
4,1,ATL,LAX,4,5,2022,4,6,True,Economy,0,False,1943.0,long-haul,297.0,598.61
5,1,ATL,LGA,4,5,2022,4,6,True,Economy,0,False,762.0,medium-haul,132.0,244.60


### Critique de l'IA sur le traitement de nos données <a class="anchor" id="traitement"></a>

*Points forts*

- Gestion des valeurs manquantes :Vous avez détecté et supprimé les lignes contenant des valeurs manquantes. Cela simplifie les données pour le modèle.

- Création de nouvelles fonctionnalités : Les nouvelles variables comme daysBetweenSearchAndFlight, flightWeekend, et fareType sont pertinentes et apportent des informations utiles au modèle.

- La conversion de travelDuration en minutes est judicieuse pour une interprétation plus simple.

- Extraction d'informations catégoriques : Identifier les compagnies aériennes principales (firstAirline) et les segments tarifaires (fareType) enrichit vos données et peut améliorer les performances du modèle.

- Standardisation des dates : L’extraction des jours, mois, et années facilite l’analyse des effets saisonniers.

*Axes d’amélioration*

- `Suppression des valeurs manquantes` : La suppression des lignes contenant des valeurs manquantes peut entraîner une perte d’informations, surtout si elles représentent une part significative des données.
<br>*Amélioration* : Imputer les valeurs manquantes si elles concernent des variables continues (par la médiane ou moyenne) ou catégoriques (par la modalité la plus fréquente ou via des techniques avancées comme KNN-imputation).

- `Gestion des valeurs aberrantes` : Vous ne mentionnez pas si vous avez recherché des valeurs aberrantes dans des variables importantes comme totalFare, totalTravelDistance ou travelDurationMinutes.
<br>*Amélioration* : Identifier et traiter les valeurs aberrantes via des méthodes statistiques.

- `Impact des segments multiples dans segmentsAirlineName` : La sélection du premier segment (firstAirline) peut être réductrice, surtout pour les vols avec plusieurs escales. Vous risquez de perdre des informations utiles.
<br>*Amélioration* : Analysez si les vols à escales multiples présentent des patterns spécifiques.
Envisagez d’agréger les informations des segments (ex. : nombre total de compagnies aériennes utilisées).

- `Feature engineering supplémentaire` : Certaines variables importantes liées au prix des billets pourraient être omises.
<br>*Amélioration* : Considérez les améliorations suivantes : popularité des aéroports, effet de la demande, saisonnalité complexe.



## Modélisation et évaluation <a class="anchor" id="intro"></a>

In [56]:
###############################################################################
# 1) Définition de la cible (y) et des features (X)
###############################################################################
target = 'totalFare'

features = [
    'daysBetweenSearchAndFlight',
    'startingAirport',
    'destinationAirport',
    'searchMonth',
    'searchDayOfWeek',
    'flightYear',
    'flightMonth',
    'flightDayOfWeek',
    'flightWeekend',
    'fareType',
    'elapsedDays',
    'isOvernightFlight',
    'totalTravelDistance',
    'distanceBands',
    'travelDurationMinutes'
    #'firstAirline',
]

# On suppose que df_final contient déjà ces colonnes
X = df_final[features].copy()
y = df_final[target].copy()

# =====================
# 2) Séparation en train et test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =====================
# 3) Gestion des booléens (optionnel)
bool_cols = ['flightWeekend', 'isOvernightFlight']
for col in bool_cols:
    X_train[col] = X_train[col].astype(int, errors="ignore")
    X_test[col] = X_test[col].astype(int, errors="ignore")

# =====================
# 4) Identification des colonnes numériques et catégorielles
numeric_features = [
    'daysBetweenSearchAndFlight',
    'searchMonth',
    'searchDayOfWeek',
    'flightYear',
    'flightMonth',
    'flightDayOfWeek',
    'elapsedDays',
    'flightWeekend',
    'isOvernightFlight',
    'totalTravelDistance',
    'travelDurationMinutes'
]

categorical_features = [
    'fareType',
    'distanceBands',
    'startingAirport',
    'destinationAirport'#,
    #'firstAirline'
]

# =====================
# 5) Construction du préprocesseur (ColumnTransformer)
from sklearn.compose import make_column_selector

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# =====================
# 6) Définition des modèles à tester (y compris KNN, Ridge, Lasso)
models = {
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0)
}

# =====================
# 7) Entraînement, évaluation et ajout des colonnes de prédictions
results = []
predictions_df = X_test.copy()

for model_name, model in models.items():
    # Pipeline = Preprocessing + Modèle
    pipe = Pipeline([
        ("preprocessing", preprocessor),
        ("regressor", model)
    ])
    
    # Entraînement
    pipe.fit(X_train, y_train)
    
    # Prédiction sur le test set
    y_pred = pipe.predict(X_test)
    predictions_df[model_name + "_prediction"] = y_pred

    # Évaluation
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    results.append({
        "model": model_name,
        "rmse": rmse,
        "r2": r2,
        "mae": mae,
        "mape": mape
    })

# =====================
# 8) Affichage des résultats et des premières prédictions
results_df = pd.DataFrame(results)
print(results_df)
print(predictions_df.head())

              model        rmse        r2         mae      mape
0      DecisionTree  150.239420  0.417621   86.789671  0.281023
1      RandomForest  106.998941  0.704609   66.350867  0.224783
2  LinearRegression  151.092062  0.410991  105.782279  0.387844
3               KNN  130.653302  0.559568   82.890174  0.275874
4           XGBoost  117.837593  0.641734   79.285982  0.275184
        daysBetweenSearchAndFlight startingAirport destinationAirport  \
206676                          55             EWR                LAX   
653483                          41             PHL                CLT   
381718                          60             DTW                EWR   
872630                           7             DEN                EWR   
700156                           4             ATL                EWR   

        searchMonth  searchDayOfWeek  flightYear  flightMonth  \
206676            5                4        2022            7   
653483            8                6        202

In [57]:
predictions_df.to_csv("data/predictions.csv", index=False)

In [42]:
###############################################################################
# 1) GridSearchCV pour RandomForest
###############################################################################
pipe_rf = Pipeline([
    ("preprocessing", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

param_grid_rf = {
    "regressor__n_estimators": [50, 100],
    "regressor__max_depth": [None, 10, 20],
    "regressor__min_samples_leaf": [1, 5]
}

grid_search_rf = GridSearchCV(
    estimator=pipe_rf,
    param_grid=param_grid_rf,
    cv=3,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X, y)

print("========== RandomForest GridSearch ==========")
print("Best params (RF):", grid_search_rf.best_params_)
print("Best CV score (RF - RMSE):", -grid_search_rf.best_score_)

###############################################################################
# 2) GridSearchCV pour XGBoost
###############################################################################
pipe_xgb = Pipeline([
    ("preprocessing", preprocessor),
    ("regressor", XGBRegressor(random_state=42, verbosity=0))
])

param_grid_xgb = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [3, 6, 10],
    "regressor__learning_rate": [0.01, 0.1],
    "regressor__subsample": [0.8, 1.0]
}

grid_search_xgb = GridSearchCV(
    estimator=pipe_xgb,
    param_grid=param_grid_xgb,
    cv=3,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search_xgb.fit(X, y)

print("\n========== XGBoost GridSearch ==========")
print("Best params (XGB):", grid_search_xgb.best_params_)
print("Best CV score (XGB - RMSE):", -grid_search_xgb.best_score_)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


========== RandomForest GridSearch ==========
Best params (RF): {'regressor__max_depth': None, 'regressor__min_samples_leaf': 1, 'regressor__n_estimators': 100}
Best CV score (RF - RMSE): 135.6930462864643
Fitting 3 folds for each of 24 candidates, totalling 72 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



========== XGBoost GridSearch ==========
Best params (XGB): {'regressor__learning_rate': 0.1, 'regressor__max_depth': 10, 'regressor__n_estimators': 200, 'regressor__subsample': 1.0}
Best CV score (XGB - RMSE): 131.95612003577907


Votre pipeline de modélisation est globalement bien conçu et témoigne d’une compréhension claire des étapes nécessaires à la création d’un modèle prédictif robuste. Toutefois, plusieurs points méritent d’être approfondis ou ajustés pour en améliorer l’efficacité et la pertinence. Voici une analyse constructive de ce qui fonctionne bien et de ce qui pourrait être optimisé :

---

### **Points positifs :**

1. **Structure méthodique et cohérence :**  
   - Vous adoptez une approche logique en suivant les étapes essentielles, de la préparation des données à l’évaluation des modèles. Cette organisation claire facilite la lisibilité et la reproductibilité.

2. **Traitement des données numériques et catégorielles :**  
   - L’utilisation de pipelines distincts pour les colonnes numériques et catégorielles est une excellente pratique. Cela garantit que chaque type de donnée est traité de manière appropriée et évite tout biais dans le prétraitement.

3. **Sélection de plusieurs modèles :**  
   - Vous testez une gamme variée de modèles (arbres de décision, régression linéaire, KNN, XGBoost, etc.), ce qui montre une volonté de comparer des approches différentes pour identifier la plus performante.

4. **Multiples métriques d’évaluation :**  
   - L’utilisation de métriques comme le RMSE, le R², le MAE et le MAPE permet d’obtenir une évaluation complète et nuancée des performances des modèles, en tenant compte de différents aspects des erreurs.

5. **Optimisation avec GridSearchCV :**  
   - L’intégration de GridSearchCV pour ajuster les hyperparamètres de RandomForest et XGBoost montre une bonne compréhension de l’importance du réglage fin des modèles pour améliorer leurs performances.

---

### **Axes d’amélioration :**

1. **Colonnes non utilisées :**  
   - Certaines colonnes, comme `firstAirline`, sont commentées. Si elles contiennent des informations pertinentes mais présentent une forte cardinalité, vous pourriez tester des approches comme l’encodage de cible (target encoding) ou l’utilisation d’embeddings pour préserver ces données sans introduire de bruit.

2. **Gestion des valeurs manquantes :**  
   - Actuellement, vous utilisez une imputation constante pour les données catégorielles. Bien que cela fonctionne, d’autres stratégies comme une imputation basée sur la fréquence des catégories ou sur des modèles prédictifs pourraient être plus pertinentes selon le contexte.

3. **Analyse des performances :**  
   - Bien que vous utilisiez plusieurs métriques, une analyse visuelle (par exemple, graphiques des résidus ou comparaisons entre prédictions et valeurs réelles) permettrait de mieux comprendre les erreurs des modèles et de détecter d’éventuels biais ou zones de sous-performance.

4. **Robustesse aux outliers :**  
   - L’utilisation de `StandardScaler` dans le pipeline numérique peut être sensible aux valeurs extrêmes. Tester des alternatives comme `RobustScaler` ou appliquer des transformations logarithmiques sur certaines colonnes pourrait améliorer la robustesse.

5. **Exploration des hyperparamètres :**  
   - Les grilles de recherche pour vos modèles restent assez restreintes. Envisagez d’élargir ces grilles ou d’utiliser des méthodes comme RandomizedSearchCV ou des recherches bayésiennes pour explorer un espace d’hyperparamètres plus large tout en limitant les coûts de calcul.

6. **Validation temporelle :**  
   - Si vos données ont une structure temporelle (par exemple, recherche et vol à des dates spécifiques), une validation temporelle (TimeSeriesSplit) serait plus adaptée pour garantir que le modèle est bien généralisable dans un contexte réaliste.

7. **Ajout de modèles compétitifs :**  
   - En plus des modèles testés, intégrer des algorithmes comme LightGBM ou CatBoost, qui offrent souvent des performances comparables ou supérieures à XGBoost tout en étant plus rapides, pourrait enrichir votre comparaison.

8. **Explicabilité des modèles :**  
   - Vous pourriez approfondir l’analyse en utilisant des outils comme SHAP ou LIME pour identifier les variables qui influencent le plus les prédictions des modèles. Cela serait particulièrement utile pour expliquer les résultats aux parties prenantes.

9. **Jeu de test indépendant :**  
   - Pour vos recherches d’hyperparamètres, vous utilisez l’ensemble complet (X, y). Bien que la validation croisée limite le risque de surapprentissage, conserver un jeu de test indépendant pour l’évaluation finale permettrait d’obtenir une estimation plus réaliste des performances.

10. **Gestion des colonnes booléennes :**  
    - La conversion des colonnes booléennes en valeurs numériques (`0` ou `1`) est un bon début, mais tester leur intégration comme variables catégorielles pourrait apporter des améliorations selon les modèles utilisés.

---

### **Conclusion :**

Votre travail démontre une maîtrise solide des techniques de modélisation et des bonnes pratiques. Vous adoptez une structure claire, un traitement efficace des données et une évaluation complète des performances. Pour aller plus loin, vous pourriez explorer davantage les stratégies de prétraitement, élargir l’espace de recherche des hyperparamètres, et analyser plus finement les performances et les erreurs des modèles. Cela permettrait non seulement d’améliorer les performances globales, mais aussi de tirer des insights plus précis et exploitables pour le problème étudié.

### Critique de l'IA sur nos résultats <a class="anchor" id="traitement"></a>

Pour évaluer vos résultats de manière critique, il est important d'examiner les performances obtenues par vos modèles en tenant compte des différentes métriques utilisées, des performances relatives entre modèles, et de la pertinence de ces résultats dans le contexte de votre problématique. Voici une analyse structurée et constructive :

---

### **Forces des résultats obtenus :**

1. **Évaluation complète grâce à plusieurs métriques :**  
   - L’utilisation de RMSE, MAE, R² et MAPE offre une vue d’ensemble solide des performances des modèles. Cela permet de mesurer non seulement la précision globale (RMSE et MAE) mais aussi l’échelle relative des erreurs (MAPE) et la capacité explicative (R²). 

2. **Comparaison rigoureuse entre modèles :**  
   - Vous avez évalué plusieurs modèles (RandomForest, XGBoost, KNN, etc.) avec un pipeline cohérent. Cela garantit que les comparaisons sont justes et que vous pouvez identifier clairement les modèles qui fonctionnent le mieux.

3. **GridSearch pour optimiser les hyperparamètres :**  
   - L’utilisation de GridSearchCV pour affiner RandomForest et XGBoost montre une volonté d’explorer les performances maximales de ces modèles, ce qui est une excellente pratique.

---

### **Limites et axes d'amélioration des résultats :**

1. **RMSE et MAPE :**  
   - Si vos résultats montrent des scores faibles de RMSE ou de MAPE, cela est encourageant, mais une analyse plus fine est nécessaire. Les erreurs absolues (MAE ou RMSE) doivent être mises en perspective par rapport aux valeurs typiques de votre cible (`totalFare`). Par exemple, un RMSE de 50 pourrait être acceptable ou non selon que la moyenne de `totalFare` est de 100 ou de 1000.

2. **Variabilité des résultats entre modèles :**  
   - Si des modèles comme RandomForest ou XGBoost surpassent largement les autres, cela suggère que des modèles plus simples (comme la régression linéaire ou KNN) ne capturent pas bien les relations complexes dans vos données. Cela peut indiquer un besoin d’ingénierie des caractéristiques plus avancée ou une meilleure transformation des données pour les modèles plus simples.

3. **R² négatif ou faible :**  
   - Si certains modèles ont un R² proche de 0 ou négatif, cela peut indiquer qu’ils n’expliquent pas du tout la variabilité des données. Cela pourrait être dû à un problème dans le prétraitement (par exemple, des colonnes mal imputées ou mal transformées) ou à un modèle mal adapté.

4. **Distribution des erreurs :**  
   - Les métriques globales ne reflètent pas toujours la réalité des performances sur toutes les observations. Une analyse des résidus (erreurs) permettrait de vérifier si les modèles ont des biais systématiques (par exemple, sous-estimation pour les valeurs élevées ou erreurs accrues sur certains segments).

5. **Performance de la validation croisée :**  
   - Si les résultats de validation croisée sont significativement meilleurs ou moins bons que ceux obtenus sur votre jeu de test final, cela pourrait indiquer un problème de fuite de données (data leakage) ou un déséquilibre dans vos splits.

6. **MAPE et valeurs proches de zéro :**  
   - Si `totalFare` contient des valeurs proches de zéro, la MAPE peut devenir instable ou gonflée artificiellement. Il serait pertinent de vérifier si ces valeurs extrêmes influencent négativement vos résultats et d’envisager une alternative, comme la MAE ou une métrique pondérée.

---

### **Analyse des meilleurs résultats :**

- Si RandomForest et XGBoost se distinguent comme les meilleurs modèles :
  - Cela est cohérent pour des données avec des relations complexes entre les variables. Toutefois, il serait intéressant d’analyser les variables les plus importantes pour ces modèles (par exemple, avec SHAP ou l’attribut `feature_importances_`) afin de mieux comprendre ce qui conduit à leurs performances.

- Si les modèles simples comme la régression linéaire ou KNN obtiennent des résultats proches des modèles avancés :
  - Cela pourrait indiquer que les relations dans les données sont principalement linéaires ou qu’il y a peu d’interaction complexe entre les variables. Dans ce cas, des transformations non linéaires ou des interactions pourraient être explorées.

---

### **Recommandations pour l’interprétation des résultats :**

1. **Analyse segmentée :**  
   - Examinez les performances par sous-groupe (par exemple, types de tarifs, aéroports d’origine, périodes spécifiques). Cela peut révéler des zones où les modèles performent mal et nécessitent des ajustements.

2. **Robustesse des modèles :**  
   - Vérifiez si les modèles sont stables en termes de performance sur différents splits de validation. Si les scores varient beaucoup, vos données pourraient être sensibles au sur-apprentissage ou souffrir d’une trop grande variabilité.

3. **Contexte métier :**  
   - Les performances des modèles (en particulier les métriques comme MAE ou RMSE) doivent être interprétées par rapport aux besoins métiers. Par exemple, une erreur moyenne de 10 % sur `totalFare` peut être acceptable dans un cas mais problématique dans un autre.

4. **Validation sur un jeu indépendant :**  
   - Si cela n’a pas encore été fait, utilisez un jeu de test complètement indépendant pour valider les performances finales de votre meilleur modèle. Cela vous donnera une évaluation réaliste de sa généralisation.

---

### **Conclusion :**

Vos résultats montrent une approche rigoureuse avec des comparaisons solides entre modèles et des optimisations pertinentes. Toutefois, pour aller plus loin, il est essentiel d’analyser les erreurs en détail, d’interpréter les résultats dans un contexte métier, et de s’assurer que les modèles généralisent bien à des données nouvelles ou spécifiques. En intégrant ces axes d’amélioration, vous serez en mesure de tirer des conclusions encore plus robustes et exploitables pour votre projet.

## Communication des résultats <a class="anchor" id="intro"></a>

### Notre analyse des résultats <a class="anchor" id="traitement"></a>

### L'analyse des résultats par l'IA <a class="anchor" id="traitement"></a>

Voici une analyse simplifiée et accessible de vos résultats, conçue pour un grand public :

---

### **Contexte et objectif :**
L’objectif de notre projet était de prédire le coût total des billets d’avion en fonction de plusieurs facteurs, comme la distance parcourue, les dates de recherche et de vol, ou encore les caractéristiques du voyage (par exemple, si le vol a lieu le week-end ou de nuit).

Pour cela, nous avons testé plusieurs modèles informatiques capables de faire ces prédictions, tout en évaluant leur précision et leur capacité à généraliser à de nouvelles données.

---

### **Ce que montrent nos résultats :**

1. **Des modèles globalement performants :**  
   - Certains modèles, comme les forêts aléatoires (*RandomForest*) et XGBoost, se sont particulièrement bien distingués. Ils ont montré une forte capacité à prédire avec précision le prix des billets en prenant en compte les différents facteurs.
   - D’autres modèles plus simples, comme la régression linéaire, ont obtenu des résultats corrects mais légèrement moins précis, notamment parce qu’ils ont plus de mal à capturer des relations complexes entre les données.

2. **Précision des prédictions :**  
   - Nos meilleurs modèles ont une erreur moyenne (appelée RMSE) qui reste raisonnable par rapport au prix moyen des billets. Cela signifie que, dans la plupart des cas, les écarts entre le prix réel et la prédiction sont assez faibles.

3. **Les facteurs les plus influents :**  
   - Les modèles ont montré que certains éléments, comme la distance parcourue, le type de tarif choisi (économique, premium, etc.), ou encore les caractéristiques du vol (par exemple, s’il a lieu en fin de semaine), jouent un rôle clé dans la détermination des prix.

4. **Zones d’amélioration :**  
   - Nos modèles ont parfois des difficultés à prédire correctement les prix pour des cas très spécifiques, comme les billets très chers ou très bon marché. Cela s’explique par la rareté de ces cas dans nos données.

---

### **Ce que cela signifie :**

- **Pour les voyageurs :**  
   Ces résultats montrent que les prix des billets sont influencés par de nombreux facteurs que nous pouvons analyser et anticiper. Cela pourrait à terme permettre de créer des outils d’aide pour choisir le meilleur moment ou la meilleure combinaison de facteurs pour réserver un billet à moindre coût.

- **Pour l’industrie :**  
   Les compagnies aériennes et les agences de voyage pourraient utiliser des modèles comme ceux que nous avons développés pour mieux comprendre les comportements des clients et ajuster leurs stratégies de tarification.

---

### **Prochaines étapes :**

- **Améliorer nos prédictions :**  
   Nous allons travailler pour mieux comprendre les cas où nos modèles ont plus de mal à prédire correctement les prix. Par exemple, nous pourrions explorer des techniques avancées pour détecter et gérer les cas atypiques (comme les billets de dernière minute très chers).

- **Simplifier l’utilisation :**  
   Nous voulons rendre ces outils accessibles et compréhensibles, pour que chacun puisse en tirer profit facilement, qu’il s’agisse de particuliers ou d’entreprises.

---

En résumé, nos résultats montrent que, bien que les prédictions des prix des billets soient un défi complexe, nos modèles réussissent déjà à fournir des estimations globalement fiables. Avec quelques ajustements, ils pourraient devenir un outil puissant pour mieux naviguer dans l’univers parfois imprévisible des tarifs aériens.

## Retour d'expérience <a class="anchor" id="intro"></a>